In [1]:
import glob
import os
import pandas as pd
import re
from unicodedata import normalize
from tqdm.auto import tqdm
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [2]:
list_all_txt_files=list(glob.glob(os.path.join(".","data","*","*","*","*.txt")))

In [3]:
len(list_all_txt_files)

44680

In [4]:
def read_file(path):
    text={"title":"","context":"","url":""}
    page_view_line = 0
    with open(path, 'r', encoding='utf-8-sig') as f:
        for n, line in enumerate(f):
            line = line.strip()
            if n == 0: # title line
                text["title"] = line.strip()
            else:
                if line:
                    if re.match(r'^[\d,]+$', line):
                        page_view_line = n
                        continue
                    if line == 'พิมพ์' or page_view_line and page_view_line < n: # skip 'print'
                        continue
                    if re.match(r'^ที่มา : http', line):
                        text["url"] = line.strip().split(" ")[-1]
                    else:
                        text["context"] +=line.strip().replace("\xa0","")+"\n"
    text["context"]=cleanhtml(normalize('NFKD', text["context"])).strip()
    text["raw"]=text["title"]+"\n"+text["context"]
    return text

In [5]:
print(read_file(list_all_txt_files[-1]))

{'title': 'รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รมช.มนัญญา ประชุมตรวจติดตามการดำเนินงานขับเคลื่อนไทยไปด้วยกันจังหวัดอุทัยธานี เพื่อพัฒนาและแก้ไขปัญหาระดับจังหวัด สามารถขับเคลื่อนไปได้โดยเร็ว', 'context': 'วันพฤหัสบดีที่ 24 ธันวาคม 2563\nรมช.มนัญญา ประชุมตรวจติดตามการดําเนินงานขับเคลื่อนไทยไปด้วยกันจังหวัดอุทัยธานี เพื่อพัฒนาและแก้ไขปัญหาระดับจังหวัด สามารถขับเคลื่อนไปได้โดยเร็ว\nรมช.มนัญญา ประชุมตรวจติดตามการดําเนินงานขับเคลื่อนไทยไปด้วยกันจังหวัดอุทัยธานี เพื่อพัฒนาและแก้ไขปัญหาระดับจังหวัด สามารถขับเคลื่อนไปได้โดยเร็ว\nนางสาวมนัญญา ไทยเศรษฐ์ รัฐมนตรีช่วยว่าการกระทรวงเกษตรและสหกรณ์ เป็นประธานการประชุมตรวจติดตามการดําเนินงานขับเคลื่อนไทยไปด้วยกันระดับพื้นที่จังหวัด (จังหวัดอุทัยธานี) ณ ห้องประชุมสะแกกรัง ศาลากลางจังหวัดอุทัยธานี พร้อมรับฟังสรุปปัญหาในแต่ละด้านที่สําคัญ โดยจังหวัดอุทัยธานีให้ความสําคัญกับแนวทางการขับเคลื่อนไทยไปด้วยกัน ให้เป็นวาระและภารกิจสําคัญของจังหวัด เพื่อพัฒนาและแก้ไขปัญหาระดับจังหวัดสามารถขับเคลื่อนไปได้โดยเร็ว ลดปัญหาและอุปสรรค รวมทั้งใช้ทรัพยากรต่างๆ ได้อย่างมีประสิทธิภ

In [6]:
title=[]
context=[]
url=[]
raw=[]
for i in tqdm(list_all_txt_files):
    d=read_file(i)
    title.append(d["title"])
    context.append(d["context"])
    url.append(d["url"])
    raw.append(d["raw"])

  0%|          | 0/44680 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame.from_dict(
    {
        "title": title,
        "context": context,
        "raw":raw,
        "url": url,
    }
)

In [8]:
df

,title,context,raw,url
0,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-โฆษกรัฐบาลแจงสิทธิ...,วันศุกร์ที่ 15 กรกฎาคม 2565\nโฆษกรัฐบาลแจงสิทธ...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-โฆษกรัฐบาลแจงสิทธิ...,https://www.thaigov.go.th/news/contents/detail...
1,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ออมสินร่วมงาน Thai...,วันพฤหัสบดีที่ 9 ธันวาคม 2564\nออมสินร่วมงาน T...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ออมสินร่วมงาน Thai...,https://www.thaigov.go.th/news/contents/detail...
2,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รัฐบาลแจงผลงานขับเ...,วันพฤหัสบดีที่ 6 มกราคม 2565\nรัฐบาลแจงผลงานขั...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รัฐบาลแจงผลงานขับเ...,https://www.thaigov.go.th/news/contents/detail...
3,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-“อนุทิน” เปิดใช้เค...,วันจันทร์ที่ 13 กันยายน 2564\n“อนุทิน” เปิดใช้...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-“อนุทิน” เปิดใช้เค...,https://www.thaigov.go.th/news/contents/detail...
4,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-“เฉลิมชัย” ประธานฟ...,วันศุกร์ที่ 15 ตุลาคม 2564\n“เฉลิมชัย” ประธานฟ...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-“เฉลิมชัย” ประธานฟ...,https://www.thaigov.go.th/news/contents/detail...
...,...,...,...,...
44675,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ออมสินแจ้งเหตุการณ...,วันพฤหัสบดีที่ 24 ธันวาคม 2563\nออมสินแจ้งเหตุ...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ออมสินแจ้งเหตุการณ...,https://www.thaigov.go.th/news/contents/detail...
44676,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล- กระทรวงเกษตรฯ จัด...,วันพฤหัสบดีที่ 24 ธันวาคม 2563\nกระทรวงเกษตรฯ ...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล- กระทรวงเกษตรฯ จัด...,https://www.thaigov.go.th/news/contents/detail...
44677,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รมว.มท. ประชุมร่วม...,วันพฤหัสบดีที่ 24 ธันวาคม 2563\nรมว.มท. ประชุม...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รมว.มท. ประชุมร่วม...,https://www.thaigov.go.th/news/contents/detail...
44678,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-อิสราเอลยังต้องการ...,วันพฤหัสบดีที่ 24 ธันวาคม 2563\nอิสราเอลยังต้อ...,รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-อิสราเอลยังต้องการ...,https://www.thaigov.go.th/news/contents/detail...


In [9]:
from datasets import Dataset

ds = Dataset.from_pandas(df)

In [10]:
ds.push_to_hub("pythainlp/thaigov-v2-corpus-31032024")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]